<a href="https://colab.research.google.com/github/skm1312/Tokyo-Olympics-Predictions/blob/main/olympics_medal_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('athletes.csv')
df.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [3]:
del df['dob'],df['id'],df['name']
df.head()

,nationality,sex,height,weight,sport,gold,silver,bronze
0,ESP,male,1.72,64.0,athletics,0,0,0
1,KOR,female,1.68,56.0,fencing,0,0,0
2,CAN,male,1.98,79.0,athletics,0,0,1
3,MDA,male,1.83,80.0,taekwondo,0,0,0
4,NZL,male,1.81,71.0,cycling,0,0,0


In [4]:
df['nationality'] = df['nationality'].astype(str)
df.isnull().sum()

nationality      0
sex              0
height         330
weight         659
sport            0
gold             0
silver           0
bronze           0
dtype: int64

In [5]:
imputer =SimpleImputer(missing_values=np.nan, strategy= 'mean')
imputer.fit(df.iloc[:, 2:4])
df.iloc[:,2:4] = imputer.transform(df.iloc[:,2:4])

In [6]:
print(len(df.nationality.unique()))
len(df.sport.unique())

207


28

In [7]:
x = df.iloc[:, 0:5].values.transpose()
y = df.iloc[:, [5,6,7]].values

In [8]:
print(x)

[['ESP' 'KOR' 'CAN' ... 'CHN' 'VIE' 'RSA']
 ['male' 'female' 'male' ... 'male' 'male' 'male']
 [1.72 1.68 1.98 ... 1.85 1.6 1.85]
 [64.0 56.0 79.0 ... 100.0 56.0 70.0]
 ['athletics' 'fencing' 'athletics' ... 'wrestling' 'weightlifting'
  'athletics']]


In [9]:
le = LabelEncoder()
x[0] = le.fit_transform(x[0])
x[4] = le.fit_transform(x[4])
x[1] = le.fit_transform(x[1])
print(x)

[[59 102 33 ... 38 202 158]
 [1 0 1 ... 1 1 1]
 [1.72 1.68 1.98 ... 1.85 1.6 1.85]
 [64.0 56.0 79.0 ... 100.0 56.0 70.0]
 [2 9 2 ... 27 26 2]]


In [10]:
gender = df['sex'].values
nationality = df['nationality'].values
sport = df['sport'].values

In [11]:
cat_gen = dict(zip(gender,x[1]))
cat_nation = dict(zip(nationality,x[0]))
cat_sport = dict(zip(sport,x[4]))
print(cat_gen)
print(cat_nation)
print(cat_sport)
full_data = [cat_nation,cat_gen,cat_sport]

{'male': 1, 'female': 0}
{'ESP': 59, 'KOR': 102, 'CAN': 33, 'MDA': 119, 'NZL': 141, 'AUS': 10, 'USA': 198, 'ETH': 61, 'BRN': 27, 'IOA': 86, 'GBR': 68, 'UZB': 199, 'RSA': 158, 'EGY': 56, 'MAR': 116, 'QAT': 155, 'SUD': 175, 'ALG': 2, 'DEN': 51, 'NED': 135, 'DJI': 52, 'SEN': 162, 'CMR': 40, 'ITA': 93, 'NIG': 138, 'SWE': 179, 'GHA': 73, 'AFG': 0, 'AZE': 12, 'KSA': 104, 'BAN': 14, 'NGR': 137, 'RUS': 159, 'IND': 85, 'HUN': 83, 'KAZ': 98, 'BDI': 16, 'ERI': 57, 'POL': 151, 'BRA': 26, 'GEO': 70, 'CZE': 50, 'SEY': 163, 'GAM': 67, 'LTU': 113, 'IRI': 87, 'ROU': 157, 'CUB': 48, 'SLO': 167, 'BAH': 13, 'ARG': 6, 'PUR': 154, 'FRA': 64, 'RWA': 160, 'TOG': 188, 'MDV': 120, 'TUN': 191, 'ISR': 91, 'LAT': 106, 'JOR': 96, 'MAS': 117, 'LIB': 111, 'LBA': 107, 'PLE': 148, 'IRQ': 89, 'TUR': 192, 'VEN': 201, 'JPN': 97, 'TPE': 189, 'KGZ': 100, 'CHN': 38, 'MEX': 121, 'GRE': 74, 'IRL': 88, 'JAM': 95, 'SUI': 176, 'BAR': 15, 'HON': 82, 'ANG': 4, 'GER': 72, 'COL': 43, 'URU': 197, 'MNE': 127, 'SRB': 171, 'BUL': 29, 'FI

In [12]:
x = x.transpose()
ranFor = RandomForestRegressor(n_estimators= 245, random_state=40,max_depth=14,min_samples_split=10)
ranFor.fit(x,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=245, n_jobs=None, oob_score=False,
                      random_state=40, verbose=0, warm_start=False)

In [13]:
def inputs_conv(country, gender, height, weight, sport):
  cnt = full_data[0].get(country.upper())
  gen = full_data[1].get(gender.lower())
  spt = full_data[2].get(sport.lower())
  hgt = height
  wt = weight
  list_all =[cnt,gen,hgt,wt,spt]
  pred = ranFor.predict([list_all])
  
  #Gold medal prediction
  if pred[0][0]<0.036:
    gold = 0
  elif pred[0][0]>0.036 and pred[0][0]<0.12:
    gold =1
  elif pred[0][0]>0.12 and pred[0][0]<0.9:
    gold =2
  elif pred[0][0]>0.9:
    gold ='2+'
  
  #Silver medal predicition
  if pred[0][1]<0.3:
    silver = 0
  elif pred[0][1]>0.3 and pred[0][1]<0.5:
    silver = 1
  elif pred[0][1]>0.5:
    silver = '1+'
  
  #Bronze medal prediciton
  if pred[0][2]<0.21:
    bronze = 0
  elif pred[0][2]>0.21 and pred[0][2]<4.5:
    bronze =1
  elif pred[0][2]>4.5:
    bronze ='1+'
  
  predict = f'Gold : {gold} \nSilver: {silver}\nBronze: {bronze}\n'
  return predict

In [15]:
nat = input('Country: ')
gend = input('Gender: ')
heigh = float(input('Height (in meters): '))
weig = float(input('Weight (in kgs): '))
spor = input('Sports: ')

print(inputs_conv(nat, gend, heigh, weig,spor))

Country: CAN
Gender: female
Height (in meters): 1.85
Weight (in kgs): 70
Sports: aquatics
Gold : 2 
Silver: 0
Bronze: 1

